<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-001.png" class="img-responsive"/>
</div>



In [ ]:
def count_chars(input_str):
    pass

count_chars('hello there')

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-002.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-003.png" class="img-responsive"/>
</div>


In [ ]:
def replace_pairs(lst, pair, idx):
    pass

print(replace_pairs([1, 2, 3, 1, 2, 3], (1, 2), 4))

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-004.png" class="img-responsive"/>
</div>





In [ ]:
import os

try:
    import google.colab
    REPO_URL = "https://github.com/wtheisen/nd-cse-10124-lectures.git"

    REPO_NAME = "/content/nd-cse-10124-lectures"
    L_PATH = "nd-cse-10124-lectures/Datasets"

    %cd /content/
    !rm -r {REPO_NAME}

    # Clone repo
    if not os.path.exists(REPO_NAME):
        !git clone {REPO_URL}

        # cd into the data folder
        %cd {L_PATH}
        !pwd

except ImportError:
    print("Unable to download repo, either:")
    print("\tA.) You're not on colab")
    print("\tB.) It has already been cloned")


import utilities as uts

In [ ]:
print('Size of Jabberwocky Graph (Words):', len(uts.build_graph_word('jabberwocky.txt')))
print('Size of Zoomer Graph (Words):', len(uts.build_graph_word('zoomer.txt')))
print('Size of Shakespeare Graph (Words):', len(uts.build_graph_word('shakespeare.txt')))
print('#' * 80)
print('Size of Jabberwocky Graph (Chars):', len(uts.build_graph_char('jabberwocky.txt')))
print('Size of Shakespeare Graph (Chars):', len(uts.build_graph_char('shakespeare.txt')))
print('Size of Zoomer Graph (Chars):', len(uts.build_graph_char('zoomer.txt')))
print('#' * 80)

In [ ]:
"""
Minimal (byte-level) Byte Pair Encoding tokenizer.

Algorithmically follows along the GPT tokenizer:
https://github.com/openai/gpt-2/blob/master/src/encoder.py
"""

class Simple_Tokenizer():
    def __init__(self):
        self.merges = {} # (int, int) -> int

        vocab = {idx: bytes([idx]) for idx in range(256)} # int -> bytes


    def _get_stats(self, ids, counts=None):
        """
        Given a list of integers, return a dictionary of counts of consecutive pairs
        Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
        Optionally allows to update an existing dictionary of counts
        """
        counts = {} if counts is None else counts

        for pair in zip(ids, ids[1:]): # iterate consecutive elements
            counts[pair] = counts.get(pair, 0) + 1

        return counts

    def _merge(self, ids, pair, idx):
        """
        In the list of integers (ids), replace all consecutive occurrences
        of pair with the new integer token idx
        Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
        """
        newids = []
        i = 0

        while i < len(ids):
            # if not at the very last position AND the pair matches, replace it
            if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1

        return newids

    def train(self, text, vocab_size, verbose=False):
        num_merges = vocab_size - 258

        # input text preprocessing
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255

        for i in range(num_merges):
            # count up the number of times every consecutive pair appears
            stats = self._get_stats(ids)

            # find the pair with the highest count
            pair = max(stats, key=stats.get)

            # mint a new token: assign it the next available id
            idx = 258 + i

            # replace all occurrences of pair in ids with idx
            ids = self._merge(ids, pair, idx)

            # save the merge
            self.merges[pair] = idx
            self.vocab[idx] = self.vocab[pair[0]] + self.vocab[pair[1]]

    def decode(self, ids):
        # given ids (list of integers), return Python string
        text_bytes = b"".join(self.vocab[idx] for idx in ids)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

    def encode(self, text):
        # given a string text, return the token ids
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255

        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = self._get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))

            # just the first pair in the list, arbitrarily
            # we can detect this terminating case by a membership check
            if pair not in self.merges:
                break # nothing else can be merged anymore

            # otherwise let's merge the best pair (lowest merge index)
            idx = self.merges[pair]
            ids = self._merge(ids, pair, idx)

        return ids

    def visualize_tokenization(self, ids):
        """Small helper function useful in debugging: visualize the tokenization of render_conversation"""
        RED = '\033[91m'
        GREEN = '\033[92m'
        RESET = '\033[0m'
        GRAY = '\033[90m'

        tokens = []
        for token_id in ids:
            token_str = self.decode([token_id])
            tokens.append(f"{GREEN}{token_str}{GRAY}({token_id}){RESET}")

            if token_str in ['<|SOS|>', '<|EOS|>']:
                tokens.append('\n\n\t')

        return ' | '.join(tokens)

In [ ]:
from tokenizer import Simple_Tokenizer

def create_token_graph(file_name):
    training_str = uts.get_file_as_string(file_name)
    testing_lines = uts.get_file_as_list_strs(file_name)

    tokenizer = Simple_Tokenizer()
    tokenizer.train(training_str, 512)

    tokenized_lines = []
    for line in testing_lines:
        tokenized_str = tokenizer.encode(line.lower())
        tokenized_lines.append([tokenizer.decode([token]) for token in tokenized_str])

    token_graph = uts.build_graph_word(tokenized_lines, file=False)

    return token_graph

print('Size of Jabberwocky Graph (Tokens):', len(create_token_graph('jabberwocky.txt')))
#print('Size of Shakespeare Graph (Tokens):', len(create_token_graph('shakespeare.txt')))

token_graph = create_token_graph('zoomer.txt')
print(token_graph)
print('Size of Zoomer Graph (Tokens):', len(token_graph))
print('#' * 80)

num_gens = 3

word_graph = uts.build_graph_word('zoomer.txt')
print('Word Level Generation [zoomer.txt]:')
for i in range(0, num_gens):
  print(f'\t{i}: {' '.join(uts.generate_sequence(word_graph, 50))}')

print('#' * 80)

char_graph = uts.build_graph_char('zoomer.txt')
print('Character Level Generation [zoomer.txt]:')
for i in range(0, num_gens):
  print(f'\t{i}: {''.join(uts.generate_sequence(char_graph, 50))}')

print('#' * 80)

print('Token Level Generation [zoomer.txt]:')
for i in range(0, num_gens):
  print(f'\t{i}: {''.join(uts.generate_sequence(token_graph, 50))}')


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-014.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-016.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-017.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-018.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-019.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-020.png" class="img-responsive"/>
</div>

In [ ]:
class Cat():
    def __init__(self, color):
        self.color = color

    # TODO

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-019.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-022.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-021.png" class="img-responsive"/>
</div>

In [ ]:
"""
Minimal (byte-level) Byte Pair Encoding tokenizer.

Algorithmically follows along the GPT tokenizer:
https://github.com/openai/gpt-2/blob/master/src/encoder.py

Unlike BasicTokenizer:
- RegexTokenizer handles an optional regex splitting pattern.
- RegexTokenizer handles optional special tokens.
"""

import regex as re

# the main GPT text split patterns, see
# https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""


class RegexTokenizer():
    def __init__(self):
        self.merges = {} # (int, int) -> int
        self.special_tokens = {'<|sos|>': 256, '<|eos|>': 257} # str -> int, e.g. {'<|endoftext|>': 100257}
        self.vocab = {idx: bytes([idx]) for idx in range(256)} | {idx: special.encode("utf-8") for special, idx in self.special_tokens.items()}
        #self.vocab = self._init_vocab() # int -> bytes

        self.pattern = GPT4_SPLIT_PATTERN
        self.compiled_pattern = re.compile(self.pattern)

    def _build_vocab(self):
        # vocab is simply and deterministically derived from merges
        vocab = {idx: bytes([idx]) for idx in range(256)} + {idx: special.encode("utf-8") for special, idx in special_tokens.items()}

        for special, idx in self.special_tokens.items():
            vocab[idx] = special.encode("utf-8")

        return vocab

    def _get_stats(self, ids, counts=None):
        """
        Given a list of integers, return a dictionary of counts of consecutive pairs
        Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
        Optionally allows to update an existing dictionary of counts
        """
        counts = {} if counts is None else counts
        for pair in zip(ids, ids[1:]): # iterate consecutive elements
            counts[pair] = counts.get(pair, 0) + 1
        return counts

    def _merge(self, ids, pair, idx):
        """
        In the list of integers (ids), replace all consecutive occurrences
        of pair with the new integer token idx
        Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
        """
        newids = []
        i = 0
        while i < len(ids):
            # if not at the very last position AND the pair matches, replace it
            if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1
        return newids

    def chunk(self, text):
        return re.findall(self.compiled_pattern, text)

    def train(self, text, max_vocab_size, verbose=False):
        pretrain_vocab_size = len(self.vocab)
        num_merges = max_vocab_size - pretrain_vocab_size

        # split the text up into text chunks
        text_chunks = re.findall(self.compiled_pattern, text)

        # input text preprocessing
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]

        # iteratively merge the most common pairs to create new tokens
        for i in range(num_merges):
            # count the number of times every consecutive pair appears
            stats = {}
            for chunk_ids in ids:
                # passing in stats will update it in place, adding up counts
                self._get_stats(chunk_ids, stats)

            # find the pair with the highest count
            pair = max(stats, key=stats.get)

            # mint a new token: assign it the next available id
            idx = pretrain_vocab_size + i

            # replace all occurrences of pair in ids with idx
            ids = [self._merge(chunk_ids, pair, idx) for chunk_ids in ids]

            # save the merge
            self.merges[pair] = idx
            self.vocab[idx] = self.vocab[pair[0]] + self.vocab[pair[1]]

    def decode(self, ids):
        # given ids (list of integers), return Python string
        part_bytes = []
        for idx in ids:
            if idx in self.vocab:
                part_bytes.append(self.vocab[idx])
            else:
                part_bytes.append('<|unk|>')

        text_bytes = b"".join(part_bytes)
        text = text_bytes.decode("utf-8", errors="replace")

        return text

    def _encode_chunk(self, text_bytes):
        # return the token ids
        # let's begin. first, convert all bytes to integers in range 0..255
        ids = list(text_bytes)

        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = self._get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            # subtle: if there are no more merges available, the key will
            # result in an inf for every single pair, and the min will be
            # just the first pair in the list, arbitrarily
            # we can detect this terminating case by a membership check
            if pair not in self.merges:
                break # nothing else can be merged anymore
            # otherwise let's merge the best pair (lowest merge index)
            idx = self.merges[pair]
            ids = self._merge(ids, pair, idx)

        return ids

    def encode(self, text):
        # otherwise, we have to be careful with potential special tokens in text
        # we handle special tokens by splitting the text
        # based on the occurrence of any exact match with any of the special tokens
        # we can use re.split for this. note that surrounding the pattern with ()
        # makes it into a capturing group, so the special tokens will be included
        special_pattern = "(" + "|".join(re.escape(k) for k in self.special_tokens) + ")"
        special_chunks = re.split(special_pattern, text)

        # now all the special characters are separated from the rest of the text
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for part in special_chunks[1:-1]:
            if part in self.special_token:
                # this is a special token, encode it separately as a special case
                ids.append(self.special_tokens[part])
            else:
                # this is an ordinary sequence, encode it normally
                text_chunks = re.findall(self.compiled_pattern, text)

                for chunk in text_chunks:
                    chunk_bytes = chunk.encode("utf-8") # raw bytes
                    chunk_ids = self._encode_chunk(chunk_bytes)
                    ids.extend(chunk_ids)

        return ids

    def visualize_tokenization(self, ids):
        """Small helper function useful in debugging: visualize the tokenization of render_conversation"""
        RED = '\033[91m'
        GREEN = '\033[92m'
        RESET = '\033[0m'
        GRAY = '\033[90m'

        tokens = []
        for token_id in ids:
            token_str = self.decode([token_id])
            tokens.append(f"{GREEN}{token_str}{GRAY}({token_id}){RESET}")

        return ' | '.join(tokens)

In [ ]:
training_text = uts.get_file_as_string('zoomer.txt')
testing_lines = uts.get_file_as_list_strs('zoomer.txt')

regex_tokenizer = RegexTokenizer()
print(regex_tokenizer.chunk(''.join(testing_lines[:5])))

# max_vocab_size = 512
# regex_tokenizer.train(training_text, max_vocab_size)

# print(f'Trained Vocab Size: {len(regex_tokenizer.vocab)}')

# for i in range(355, 360):
#     print('-' * 80)
#     print(testing_lines[i])

#     tokenized_str = regex_tokenizer.encode(testing_lines[i])
#     print(regex_tokenizer.visualize_tokenization(tokenized_str))

#save_markov_graph_to_json(build_graph_word(uts.get_lines_str('zoomer.txt')), "zoomer_char_graph.json")

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-023.png" class="img-responsive"/>
</div>

In [ ]:
!pip install rustbpe

# -----------------------------------------------------------------------------
# Tokenizer based on rustbpe + tiktoken combo
import rustbpe
import tiktoken

SPECIAL_TOKENS = [
    "<|sos|>", # every document begins with the Start of Sequence (SOS) token that delimits documents
    "<|soum|>", # start of user message
    "<|eoum|>", # end of user message
    "<|soam|>", # start of assistant message
    "<|eoam|>", # end of assistant message
]

SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,2}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

class RustBPETokenizer:
    def __init__(self, enc, bos_token):
        self.enc = enc
        self.bos_token_id = self.encode_special(bos_token)

    @classmethod
    def train_from_iterator(cls, text_iterator, vocab_size):
        # 1) train using rustbpe
        tokenizer = rustbpe.Tokenizer()

        # the special tokens are inserted later in __init__, we don't train them here
        vocab_size_no_special = vocab_size - len(SPECIAL_TOKENS)
        tokenizer.train_from_iterator(text_iterator, vocab_size_no_special, pattern=SPLIT_PATTERN)

        # 2) construct the associated tiktoken encoding for inference
        pattern = tokenizer.get_pattern()
        mergeable_ranks_list = tokenizer.get_mergeable_ranks()
        mergeable_ranks = {bytes(k): v for k, v in mergeable_ranks_list}
        tokens_offset = len(mergeable_ranks)
        special_tokens = {name: tokens_offset + i for i, name in enumerate(SPECIAL_TOKENS)}

        enc = tiktoken.Encoding(name="rustbpe", pat_str=pattern, mergeable_ranks=mergeable_ranks, special_tokens=special_tokens)

        return cls(enc, "<|sos|>")

    def encode(self, text):
        return self.enc.encode_ordinary(text)

    def encode_special(self, text):
        return self.enc.encode_single_token(text)

    def decode(self, ids):
        return self.enc.decode(ids)

    def render_conversation(self, conversation, max_tokens=2048):
        """
        Tokenize a single Chat conversation (which we call a "doc" or "document" here).
        Returns:
        - ids: list[int] is a list of token ids of this rendered conversation
        """
        messages = conversation["messages"]

        # now we can tokenize the conversation
        ids = [self.encode_special('<|sos|>')]

        for i, message in enumerate(messages):
            content = message["content"]

            if message["role"] == "user":
                ids += [self.encode_special('<|soum|>')] + self.encode(content) + [self.encode_special('<|eoum|>')]

            elif message["role"] == "assistant":
                ids += [self.encode_special('<|soam|>')] + self.encode(content) + [self.encode_special('<|eoam|>')]

        # truncate to max_tokens tokens MAX (helps prevent OOMs)
        ids = ids[:max_tokens]

        return ids

    def visualize_tokenization(self, ids):
        """Small helper function useful in debugging: visualize the tokenization of render_conversation"""
        RED = '\033[91m'
        GREEN = '\033[92m'
        RESET = '\033[0m'
        GRAY = '\033[90m'

        tokens = []
        for i, token_id in enumerate(ids):
            token_str = self.decode([token_id])
            tokens.append(f"{GREEN}{token_str}{GRAY}({token_id}){RESET}")

            if token_str in ['<|sos|>', '<|eoum|>', '<|eoam|>']:
                tokens.append('\n\n\t')

        return ' | '.join(tokens)

In [ ]:
training_text = uts.get_file_str('shakespeare.txt')

nanochat_tokenizer = RustBPETokenizer.train_from_iterator(training_text, 256 + len(SPECIAL_TOKENS))

conversation = {
    'messages': [
        {'role': 'user', 'content': 'Hello there general kenobi'},
        {'role': 'assistant', 'content': 'Hello there grevious'},
        {'role': 'user', 'content': 'I love neffer'},
    ]
}

ids = nanochat_tokenizer.render_conversation(conversation)
print(nanochat_tokenizer.visualize_tokenization(ids))

## Export to HTML

Uncomment the final line of the cell below and run it to export this notebook to HTML

In [ ]:
import os, json

def export_notebook():
  L_PATH = "nd-cse-10124-lectures/Notebooks"
  L = "Lecture_03_Tokenization"

  try:
      from google.colab import _message, files

      # where you WANT it to live (repo folder)
      repo_ipynb_path = f"/content/{L_PATH}/{L}.ipynb"

      # grab current notebook contents from the UI
      nb = _message.blocking_request("get_ipynb", timeout_sec=1)["ipynb"]

      # write it into the repo folder as a real file
      os.makedirs(os.path.dirname(repo_ipynb_path), exist_ok=True)
      with open(repo_ipynb_path, "w", encoding="utf-8") as f:
          json.dump(nb, f)

      # convert + download pdf
      !jupyter nbconvert --to html "{repo_ipynb_path}"
      files.download(repo_ipynb_path.replace(".ipynb", ".html"))
  except:
      import subprocess

      nb_fp = os.getcwd() + f'{L}.ipynb'
      print(os.getcwd())

      subprocess.run(["jupyter", "nbconvert", "--to", "html", nb_fp], check=True)

#export_notebook()